In [9]:
# Importing basic necessary libraries for music generation
import sys
import os
import time
import ipykernel
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, TimeDistributed, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, History
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from midi_parser import getData, createTrainData
import tensorflow as tf

In [3]:
highest_note = 81 # A_6  Value directly taken from parser.py
lowest_note = 33 # A_2   Value directly taken from parser.py
pitch_dimension = highest_note - lowest_note + 1

# Model parameters
num_hidden = 512
x_len = 100
y_len = 10
batch_size = 64
epochs = 3

load_weight = True
weight_path = "./model/Weight_LSTM.hdf5"   # Path for the weight of the model
model_path = "./model/Model_LSTM.json"     # Path for the json file of the model

In [1]:
print("Enter Genre to train = ")           # User is asked to enter the genre of the music which needs to be trained.
data_path = input()

Enter Genre to train = 
C:/Users/shivam/Documents/Music_Generator_Phase2/MusicGeneration/Classical


In [5]:
def buildModel():
    
    #Building a LSTM model

    #encoding
    model = Sequential()
    model.add(LSTM(num_hidden, input_dim = pitch_dimension, return_sequences = True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(LSTM(num_hidden))
    model.add(RepeatVector(y_length))
    
    #decoder
    model.add(LSTM(num_hidden, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(num_hidden, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(num_hidden, return_sequences = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(pitch_dimension, activation= 'softmax')))
    model.add(TimeDistributed(Dense(pitch_dimension, activation= 'softmax')))

    return model



In [ ]:
if __name__ == '__main__':
    
    # preparing data for training
    pianoroll = getData(data_path)
    X,Y = createTrainData(pianoroll, x_len, y_len)
    
    # building a model
    model = buildModel()
    model.summary()
    if load_weight:
        model.load_weight(weight_path)
    model.compile(loss='categorical_crossentropy', optimizer = RMSprop())

    # model callbacks
    checkpoint = ModelCheckpoint(weight_path, monitor='loss', verbose=0, save_best_only=True, mode='auto') # saving our model weights
    earlystop = EarlyStopping(monitor='loss', patience= 10, verbose=0, mode= 'auto') # terminating our training
    history = History() # ploting any training loss

    # train the model
    hist = model.fit(X.astype(np.bool), Y.astype(np.bool), batch_size=batch_size, epochs=num_epochs, callbacks=[earlystop, history, checkpoint])

    # save trained model structure
    open(model_path, 'w').write(model.to_json())

    # plot training loss
    img = plt.figure(figsize=(6,5), dpi=75)
    plt.plot(hist.history['loss'])
    img.savefig("TrainingLoss.png", bbox_inches='tight')